## Schwingungsgleichung des gedämpften Federpendels

Für die d'Alembertsche Trägheitskraft des Pendelkörpers mit Masse $m$ unter der Beschleunigung $\ddot{y}$ gilt
$$ F_T = -m\ddot{y}. $$
Auf den Pendelkörper wirken die als äußere Kräfte
\begin{align*}
F_F &= -Dy & &\text{Federkraft (Hook'sches Gesetz)} \\
F_{R} &= -k\dot{y} & & \text{Viskose Reibungskraft}
\end{align*}
Unter Anwendung des zweiten Axioms von Newton (Grundgleichung der Mechanik) folgt
$$ -m\ddot{y} = F_T = - (F_F + F_R) = Dy + k\dot{y} $$
Hieraus ergibt sich die Schwingungsgleichung des Federpendels
$$\ddot{y} + \frac{k}{m}\dot{y} + \frac{D}{m}y = 0.$$
Unter Einführung der Variablen
\begin{align*}
y_1 &= y\\
y_2 &= \dot{y}
\end{align*}
kann man die Schwingungsgleichung als DGL-System erster Ordnung schreiben
\begin{align*}
\dot{y}_1 &= y_2 \\
\dot{y}_2 &=  - \frac{D}{m}y_1 - \frac{k}{m}y_2
\end{align*}
oder kurz
$$\dot{\vec y} = A\vec y$$
mit
$$A = \begin{pmatrix} 0 & 1 \\ - \frac{D}{m} & - \frac{k}{m} 	\end{pmatrix} $$

In [10]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

# Module für komplexe Zahlen, Vektoren/Matrizen, gewöhnliche DGL und Grafikausgabe
import cmath
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Schwingungsgleichung für die Auslenkung y des harmonischen Oszillators
#   d^2y/dt^2 + k/m dy/dt + D/m y = 0

# Parameter der Schwingungsgleichung:
#   - Federkonstante D in N/m
#   - Masse m in kg
#   - viskoser Reibungskoeffizient k in kg/s

# Anfangsbedingung:
#   - Auslenkung y0 in m
#   - Geschwindigkeit y0dot in m/s

# Methoden zum Lösen der Schwingungsgleichung:
#   - numerisch:       Verwendung von odeint
#   - analytisch:      Verwendung der Formeln aus der Vorlesung
#   - alle:            Vergleich der Ergebnisse

def f(D=1,m=1,k=0.4,y0=1,y0dot=0,Methode='alle'):
  
  # DGL System für das gedämpfte Federpendel
  # y     = (y1, y2)
  # A     = (  0  ,   1 )
  #         (-D/m , -k/m)
  # dy/dt = A*y
  A=np.array([[0, 1],[-D/m,-k/m]])
  def dy(y,t):
    return A.dot(y)
  
  # Hilfsparameter
  delta = k/(2*m)

  # Zeitdiskretisierung: 40 Zeitschritte von 0 bis 10 s
  t = np.linspace(0,10,40)  
  
  if (Methode == 'alle' or Methode == 'numerisch'):
    # Numerische Lösung des Anfangswertproblems
    # dy/dt = A*y
    #  y(0) = [y0,y0dot] 
    y = odeint(func=dy, y0=[y0,y0dot], t=t)

  if (Methode == 'alle' or Methode == 'analytisch'):
    # Verwendung der Formeln aus der Vorlesung
    lambdaAna = [-delta + cmath.sqrt(delta**2 - D/m), -delta - cmath.sqrt(delta**2 - D/m)]
    vlambdaAna = [[1,1],[lambdaAna[0],lambdaAna[1]]]

  # Visualisierung von Eigenwerten und Schwingungsverhalten des Federpendels
  plt.figure(figsize=(16, 4))
  gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])
  plt.subplots_adjust(wspace=0.2)
    
  # 1. Visualisierung der Eigenwerte
  ax1 = plt.subplot(gs[0])
  ax1.grid(True)
  ax1.set_title('Eigenwerte')
  plt.ylabel('Imaginärteil')
  plt.xlabel('Realteil')
  if (Methode == 'alle' or Methode == 'analytisch'):
    xmax = max(abs(np.real(lambdaAna)))
    ymax = max(abs(np.imag(lambdaAna)))
    xymax = max(xmax,ymax)
    ax1.set(xlim=(-1.2*xymax, 1.2*xymax), ylim=(-1.2*xymax, 1.2*xymax))
    for v in lambdaAna:
      ax1.annotate('', xy=[v.real,v.imag], xytext=(0, 0),arrowprops=dict(facecolor='blue',shrink=0,alpha=0.6,width=0.5))

  # 2. Visualisierung des Schwingungsverhaltens
  ax2 = plt.subplot(gs[1])
  ax2.grid(True)
  ax2.set_title('Schwingungsverhalten')
  plt.ylabel('Auslenkung / m')
  plt.xlabel('Zeit / s')

  # 2. a) Ausgabe der numerischen Lösung
  if (Methode == 'alle' or Methode == 'numerisch'): 
    ax2.plot(t, y[:,0],"r.",label="numerisch")
   
  # 2. b) Ausgabe der analytischen Lösungen
  if (Methode == 'alle' or Methode == 'analytisch'):
    if (delta**2 != D/m): # schwache oder starke Dämpfung
      # Analytisch berechnete Koeffizienten cAna = [c1,c2] zum Erfüllen der Anfangsbedingung (Formeln aus der Vorlesungs)
      cAna = [-lambdaAna[1]*y0/(lambdaAna[0]-lambdaAna[1]), lambdaAna[0]*y0/(lambdaAna[0]-lambdaAna[1])]
      ax2.plot(t, (cAna[0]*np.exp(lambdaAna[0]*t)+cAna[1]*np.exp(lambdaAna[1]*t)).real,'g-', label="analytisch")
    else: # kritische Dämpfung
      ax2.plot(t, ((y0+delta*y0*t)*np.exp(lambdaAna[0]*t)),'g', label="analytisch")    
  ax2.legend()
  
  plt.show()


In [11]:
oszillator=interactive(f,D=(0,5,0.1),m=(0.1,5,0.1),k=(0,5,0.1),y0=(0,5,0.1),y0dot=(-5,5,0.1),Methode=['numerisch','analytisch','alle'])
display(oszillator)

interactive(children=(FloatSlider(value=1.0, description='D', max=5.0), FloatSlider(value=1.0, description='m'…